![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# 8. Generic Classifier v2.7.0

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark


SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 spark://10.164.245.220:7077 
 AppName 
 Databricks Shell

## Load dataset

In [0]:
 !wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/petfinder-mini.csv
 
 dbutils.fs.cp("file:/databricks/driver/petfinder-mini.csv", "dbfs:/") 

Out[2]: True

In [0]:

dataframe = pd.read_csv('petfinder-mini.csv')

In [0]:
# In the original dataset "4" indicates the pet was not adopted.

dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

In [0]:
dataframe = dataframe.drop(['AdoptionSpeed'], axis=1)

In [0]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,target
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He is energetic and playful. I rescued a couple of ca...,1,1
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartment. It was shaking so I had to bring it home to p...,2,1
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresponsible owner at the roadside near some shops in S...,7,1
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience waiting for her good master, plz call or sms for m...",8,1
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption. He is the most playful pal we've seen in our pupp...,3,1


In [0]:
dataframe.columns

Out[67]: Index(['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'Description',
 'PhotoAmt', 'target'],
 dtype='object')

In [0]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11537 entries, 0 to 11536
Data columns (total 15 columns):
Type 11537 non-null object
Age 11537 non-null int64
Breed1 11537 non-null object
Gender 11537 non-null object
Color1 11537 non-null object
Color2 11537 non-null object
MaturitySize 11537 non-null object
FurLength 11537 non-null object
Vaccinated 11537 non-null object
Sterilized 11537 non-null object
Health 11537 non-null object
Fee 11537 non-null int64
Description 11537 non-null object
PhotoAmt 11537 non-null int64
target 11537 non-null int64
dtypes: int64(4), object(11)
memory usage: 1.3+ MB

In [0]:
dataframe.target.value_counts()

Out[69]: 1 8457
0 3080
Name: target, dtype: int64

In [0]:
dataframe.Description = dataframe.Description.fillna('- no description -')

## Featurize with Sklearn Column Transformer

In [0]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

column_trans = make_column_transformer(
     (OneHotEncoder(handle_unknown='ignore', categories='auto'), ['Type', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health']),
     (TfidfVectorizer(max_features=100,  norm='l2', ngram_range=(1, 3)), 'Description'),
     remainder=StandardScaler())

X = column_trans.fit_transform(dataframe.drop(['target'], axis=1))

y = dataframe.target

/databricks/python/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
 return self.partial_fit(X, y)
/databricks/python/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
 return self.fit(X, **fit_params).transform(X)

In [0]:
y.nunique()


Out[125]: 2

In [0]:
X.shape

Out[105]: (11537, 302)

In [0]:
input_dim = X.shape[1]

In [0]:
input_dim

Out[107]: 302

In [0]:
import scipy.sparse

df = pd.SparseDataFrame(X)

df.columns = ['col_{}'.format(i) for i in range(input_dim)]

df = df.fillna(0)

df['target']= y

df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,...,col_253,col_254,col_255,col_256,col_257,col_258,col_259,col_260,col_261,col_262,col_263,col_264,col_265,col_266,col_267,col_268,col_269,col_270,col_271,col_272,col_273,col_274,col_275,col_276,col_277,col_278,col_279,col_280,col_281,col_282,col_283,col_284,col_285,col_286,col_287,col_288,col_289,col_290,col_291,col_292,col_293,col_294,col_295,col_296,col_297,col_298,col_299,col_300,col_301,target
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.171825,0.0,0.137765,0.0,0.000000,0.283609,0.0,0.220992,0.151594,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.144907,0.000000,0.0,0.158869,0.0,0.0,0.000000,0.170202,0.000000,0.192465,0.190783,0.000000,0.132427,0.176639,0.151246,0.000000,0.0,0.000000,0.000000,0.257843,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.452479,0.950288,-0.829762,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.228484,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.256152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250843,0.000000,0.0,0.000000,0.000000,0.213817,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.555981,-0.299388,-0.511871,1
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.141649,0.000000,0.0,0.000000,0.0,0.172449,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.307225,0.0,0.178193,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.210999,0.209155,0.209619,0.000000,0.000000,0.414527,0.179294,0.0,0.000000,0.000000,0.141336,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.555981,-0.299388,1.077583,1
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.257886,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.198538,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.226480,0.0,-0.400729,1.575125,1.395473,1
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.126763,0.113263,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.081638,0.000000,0.105362,0.128186,0.0,0.000000,0.0,0.000000,0.107062,0.086649,0.0,0.000000,0.0,0.0,0.000000,0.125751,0.218471,0.142200,0.000000,0.000000,0.195683,0.000000,0.111746,0.120832,0.0,0.120993,0.000000,0.000000,0.223069,0.117846,0.0,0.198113,0.0,0.133816,0.181835,0.0,-0.555981,-0.299388,-0.193980,1


## Train with Spark NLP Generic Classifier

**Building a pipeline**

The FeaturesAssembler is used to collect features from different columns. It can collect features from single value columns (anything which can be cast to a float, if casts fails then the value is set to 0), array columns or SparkNLP annotations (if the annotation is an embedding, it takes the embedding, otherwise tries to cast the 'result' field). The output of the transformer is a FEATURE_VECTOR annotation (the numeric vector is in the 'embeddings' field).

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations. The operation of the classifier is controled by the following methods:

*setEpochsNumber(int)* - Determines how many epochs the model is trained.

*setBatchSize(int)* - Sets the batch size during training.

*setLearningRate(float)* - Sets the learning rate.

*setValidationSplit(float)* - Sets the proportion of examples in the training set used for validation.

*setModelFile(string)* - Loads a model from the specified location and uses it instead of the default model.

*setFixImbalance(boolean)* - If set to true, it tries to balance the training set by weighting the classes according to the inverse of the examples they have.

*setFeatureScaling(string)* - Normalizes the feature factors using the specified method ("zscore", "minmax" or empty for no normalization).

*setOutputLogsPath(string)* - Sets the path to a folder where logs of training progress will be saved. No logs are generated if no path is specified.

In [0]:
spark_df = spark.createDataFrame(df)
spark_df.select(spark_df.columns[-10:]).show(2)

/databricks/spark/python/pyspark/sql/session.py:776: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Did not pass numpy.dtype object
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
col_293|col_294|col_295|col_296|col_297|col_298| col_299| col_300| col_301|target|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|-0.4524794726808656| 0.9502875792756131|-0.8297616989552165| 1|
 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| -0.555981017719065|-0.29938816657135553|-0.5118709929431844| 1|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
only showing top 2 rows

In [0]:

(training_data, test_data) = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 9246
Test Dataset Count: 2291

## Create a custom DL architecture with TF

In [0]:
%sh cd /databricks/python/lib/python3.7/site-packages/sparknlp_jsl/_tf_graph_builders/ && ls -la

total 40
drwxr-xr-x 7 root root 4096 Nov 1 11:55 .
drwxr-xr-x 4 root root 4096 Nov 1 11:55 ..
drwxr-xr-x 3 root root 4096 Nov 1 11:55 assertion_dl
drwxr-xr-x 3 root root 4096 Nov 1 11:55 generic_classifier
-rw-r--r-- 1 root root 9396 Nov 1 11:55 graph_builders.py
-rw-r--r-- 1 root root 0 Nov 1 11:55 __init__.py
drwxr-xr-x 3 root root 4096 Nov 1 11:55 ner_dl
drwxr-xr-x 2 root root 4096 Nov 1 11:55 __pycache__
drwxr-xr-x 3 root root 4096 Nov 1 11:55 text2sql

In [0]:
import tensorflow as tf

tf.__version__

Out[80]: '1.15.0'

In [0]:
from sparknlp_jsl.training import tf_graph

#!mkdir gc_graph

tf_graph.print_model_params("generic_classifier")


generic_classifier parameters.
Parameter Required Default value Description
hidden_layers no [200] List of integers indicating the size of each hidden layer. For example: [100, 200, 300].
input_dim yes - Input dimension.
output_dim yes - Output dimension.
hidden_act no relu Activation function of hidden layers: relu, sigmoid, tanh or linear.
hidden_act_l2 no 0 L2 regularization of hidden layer activations. Boolean (0 or 1).
hidden_weights_l2 no 0 L2 regularization of hidden layer weights. Boolean (0 or 1).
batch_norm no 0 Batch normalization. Boolean (0 or 1).

In [0]:
DL_params = {"input_dim": input_dim, 
             "output_dim": y.nunique(), 
             "hidden_layers": [300, 200, 100], 
             "hidden_act": "tanh",
             'hidden_act_l2':1,
             'batch_norm':1}


tf_graph.build("generic_classifier",build_params=DL_params, model_location="file:/databricks/driver/gc_graph", model_filename="auto")

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/ops/losses/losses_impl.py:121: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
generic_classifier graph exported to file:/databricks/driver/gc_graph/gcl.302.2.pb

In [0]:
%sh pwd && ls -la

/databricks/driver
total 28292
drwxr-xr-x 1 root root 4096 Nov 1 13:10 .
drwxr-xr-x 1 root root 4096 Nov 1 11:54 ..
-rw-r--r-- 1 root root 514752 Nov 1 12:56 amazon_alexa.tsv
drwxr-xr-x 2 root root 4096 Jan 1 1970 conf
-rw-r--r-- 1 root root 717 Nov 1 11:54 derby.log
drwxr-xr-x 3 root root 4096 Nov 1 11:54 eventlogs
drwxr-xr-x 3 root root 4096 Nov 1 12:03 file:
drwxr-xr-x 2 root root 4096 Nov 1 13:10 ganglia
drwxr-xr-x 2 root root 4096 Nov 1 12:32 gc_graph
drwxr-xr-x 2 root root 4096 Nov 1 13:00 logs
-rw-r--r-- 1 root root 4763125 Nov 1 11:57 petfinder-mini.csv
-rw-r--r-- 1 root root 4763125 Nov 1 12:50 petfinder-mini.csv.1
-rw-r--r-- 1 root root 4763125 Nov 1 13:05 petfinder-mini.csv.2
-rw-r--r-- 1 root root 4763125 Nov 1 13:10 petfinder-mini.csv.3
-rw-r--r-- 1 root root 9363435 Apr 9 2020 pubmed_sample_text_small.csv

In [0]:
#or just use the one we already have in the rpo
#! mkdir /databricks/driver/gc_graph

#!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/pet.in1202D.out2.pb -P /databricks/driver/gc_graph

In [0]:
%sh cd file:/databricks/driver/gc_graph && ls -la

total 144
drwxr-xr-x 2 root root 4096 Nov 1 13:11 .
drwxr-xr-x 3 root root 4096 Nov 1 12:03 ..
-rw-r--r-- 1 root root 135651 Nov 1 13:11 gcl.302.2.pb

In [0]:
from sparknlp_jsl.base import *

!mkdir logs

features_asm = FeaturesAssembler()\
      .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
      .setOutputCol("features")
        
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("target")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")\
    .setModelFile('file:/databricks/driver/gc_graph/gcl.302.2.pb')\
    .setEpochsNumber(100)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.001)\
    .setOutputLogsPath("logs")\
    .setValidationSplit(0.2) # keep 20% of the data for validation purposes

clf_Pipeline = Pipeline(stages=[
    features_asm, 
    gen_clf])



In [0]:
# train 100 epochs (takes around 37 seconds)

clf_model = clf_Pipeline.fit(training_data)


In [0]:
%sh cd logs && ls -lt

total 17068
-rw-r--r-- 1 root root 550215 Nov 1 13:59 active.log
-rw-r--r-- 1 root root 2365012 Nov 1 13:59 log4j-active.log
-rw-r--r-- 1 root root 9902289 Nov 1 13:59 usage.json
-rw-r--r-- 1 root root 7748 Nov 1 13:59 GenericClassifierApproach_8252c3aa4166.log
-rw-r--r-- 1 root root 177126 Nov 1 13:59 stdout
-rw-r--r-- 1 root root 1848836 Nov 1 13:59 metrics.json
-rw-r--r-- 1 root root 514529 Nov 1 13:58 stderr
-rw-r--r-- 1 root root 7740 Nov 1 13:54 GenericClassifierApproach_7d503a73b1e8.log
-rw-r--r-- 1 root root 1520 Nov 1 13:44 GenericClassifierApproach_44651382b093.log
-rw-r--r-- 1 root root 2523 Nov 1 13:22 GenericClassifierApproach_58a897c40de6.log
-rw-r--r-- 1 root root 1543 Nov 1 13:11 GenericClassifierApproach_20ac11050ad2.log
-rw-r--r-- 1 root root 1313954 Nov 1 13:00 2020-11-01-12.metrics.json.gz
-rw-r--r-- 1 root root 17796 Nov 1 13:00 2020-11-01-12.log.gz
-rw-r--r-- 1 root root 24572 Nov 1 13:00 log4j-2020-11-01-12.log.gz
-rw-r--r-- 1 root root 138449 Nov 1 13:00 2020-11-01-12.usage.json.gz
-rw-r--r-- 1 root root 33053 Nov 1 12:58 stderr--2020-11-01--13-00
-rw-r--r-- 1 root root 32334 Nov 1 12:57 stdout--2020-11-01--13-00
-rw-r--r-- 1 root root 1539 Nov 1 12:53 GenericClassifierApproach_923cbf3f3a07.log
-rw-r--r-- 1 root root 25694 Nov 1 12:00 log4j-2020-11-01-11.log.gz
-rw-r--r-- 1 root root 111500 Nov 1 12:00 2020-11-01-11.metrics.json.gz
-rw-r--r-- 1 root root 5332 Nov 1 12:00 2020-11-01-11.log.gz
-rw-r--r-- 1 root root 43179 Nov 1 12:00 2020-11-01-11.usage.json.gz
-rw-r--r-- 1 root root 281826 Nov 1 11:59 stdout--2020-11-01--12-00
-rw-r--r-- 1 root root 1076 Nov 1 11:56 stderr--2020-11-01--12-00

In [0]:
%sh

cat logs/GenericClassifierApproach_8252c3aa4166.log

Training 100 epochs
Epoch 1/100	0.47s	Loss: 19.19971	ACC: 0.6294313	Validation ACC: 0.6454054
Epoch 2/100	0.31s	Loss: 16.520258	ACC: 0.70470715	Validation ACC: 0.65243244
Epoch 3/100	0.31s	Loss: 14.934163	ACC: 0.74336714	Validation ACC: 0.70594597
Epoch 4/100	0.31s	Loss: 13.288931	ACC: 0.7848593	Validation ACC: 0.6978378
Epoch 5/100	0.31s	Loss: 11.489869	ACC: 0.82176256	Validation ACC: 0.69297296
Epoch 6/100	0.31s	Loss: 9.818456	ACC: 0.85358685	Validation ACC: 0.7194595
Epoch 7/100	0.31s	Loss: 7.972945	ACC: 0.891008	Validation ACC: 0.7313514
Epoch 8/100	0.31s	Loss: 6.2388244	ACC: 0.91859776	Validation ACC: 0.7194595
Epoch 9/100	0.31s	Loss: 4.688488	ACC: 0.94713396	Validation ACC: 0.7178378
Epoch 10/100	0.31s	Loss: 3.7823398	ACC: 0.95873314	Validation ACC: 0.71675676
Epoch 11/100	0.31s	Loss: 2.9235892	ACC: 0.9695721	Validation ACC: 0.74054056
Epoch 12/100	0.31s	Loss: 2.285054	ACC: 0.97944856	Validation ACC: 0.72864866
Epoch 13/100	0.31s	Loss: 1.7953365	ACC: 0.98458934	Validation ACC: 0.7448649
Epoch 14/100	0.33s	Loss: 1.4117141	ACC: 0.9883673	Validation ACC: 0.7308108
Epoch 15/100	0.31s	Loss: 1.26818	ACC: 0.98810834	Validation ACC: 0.73783785
Epoch 16/100	0.31s	Loss: 1.0156454	ACC: 0.9914867	Validation ACC: 0.72864866
Epoch 17/100	0.31s	Loss: 1.1237292	ACC: 0.9894597	Validation ACC: 0.7524324
Epoch 18/100	0.31s	Loss: 0.82544506	ACC: 0.9931025	Validation ACC: 0.74216217
Epoch 19/100	0.31s	Loss: 0.64183605	ACC: 0.9941894	Validation ACC: 0.7464865
Epoch 20/100	0.31s	Loss: 0.56511176	ACC: 0.9958109	Validation ACC: 0.7535135
Epoch 21/100	0.31s	Loss: 0.5136497	ACC: 0.99485964	Validation ACC: 0.734054
Epoch 22/100	0.31s	Loss: 0.5229202	ACC: 0.99472445	Validation ACC: 0.7383784
Epoch 23/100	0.31s	Loss: 0.6450801	ACC: 0.99431896	Validation ACC: 0.7464865
Epoch 24/100	0.31s	Loss: 0.35854575	ACC: 0.99702734	Validation ACC: 0.73945946
Epoch 25/100	0.31s	Loss: 0.4398248	ACC: 0.99540555	Validation ACC: 0.74702704
Epoch 26/100	0.31s	Loss: 0.8599915	ACC: 0.9886431	Validation ACC: 0.7481081
Epoch 27/100	0.31s	Loss: 0.99157286	ACC: 0.9886432	Validation ACC: 0.7362162
Epoch 28/100	0.31s	Loss: 0.610228	ACC: 0.9939191	Validation ACC: 0.7427027
Epoch 29/100	0.31s	Loss: 0.56712073	ACC: 0.9951353	Validation ACC: 0.74108106
Epoch 30/100	0.31s	Loss: 0.51734185	ACC: 0.9948595	Validation ACC: 0.7254054
Epoch 31/100	0.31s	Loss: 0.56181204	ACC: 0.9943188	Validation ACC: 0.74216217
Epoch 32/100	0.31s	Loss: 0.8202431	ACC: 0.99175704	Validation ACC: 0.72324324
Epoch 33/100	0.31s	Loss: 0.35822126	ACC: 0.9970271	Validation ACC: 0.7448649
Epoch 34/100	0.31s	Loss: 0.34087968	ACC: 0.9966217	Validation ACC: 0.73513514
Epoch 35/100	0.31s	Loss: 0.33163184	ACC: 0.99689186	Validation ACC: 0.73675674
Epoch 36/100	0.31s	Loss: 0.3496584	ACC: 0.99648666	Validation ACC: 0.74
Epoch 37/100	0.31s	Loss: 0.3202789	ACC: 0.9964865	Validation ACC: 0.7454054
Epoch 38/100	0.31s	Loss: 0.23269512	ACC: 0.9975619	Validation ACC: 0.73945946
Epoch 39/100	0.31s	Loss: 0.23390129	ACC: 0.99756753	Validation ACC: 0.7427027
Epoch 40/100	0.31s	Loss: 0.16286722	ACC: 0.9981082	Validation ACC: 0.74702704
Epoch 41/100	0.31s	Loss: 0.16071443	ACC: 0.9981081	Validation ACC: 0.7475676
Epoch 42/100	0.31s	Loss: 0.14621103	ACC: 0.9983786	Validation ACC: 0.7464865
Epoch 43/100	0.31s	Loss: 0.15064286	ACC: 0.9982433	Validation ACC: 0.74864864
Epoch 44/100	0.31s	Loss: 0.20884223	ACC: 0.9977027	Validation ACC: 0.73945946
Epoch 45/100	0.31s	Loss: 0.2635475	ACC: 0.99743253	Validation ACC: 0.7491892
Epoch 46/100	0.31s	Loss: 0.2703808	ACC: 0.9967568	Validation ACC: 0.734054
Epoch 47/100	0.31s	Loss: 1.5240077	ACC: 0.98080546	Validation ACC: 0.71567565
Epoch 48/100	0.31s	Loss: 3.6156068	ACC: 0.9548141	Validation ACC: 0.72432435
Epoch 49/100	0.31s	Loss: 2.6435847	ACC: 0.97146434	Validation ACC: 0.7075676
Epoch 50/100	0.31s	Loss: 1.179878	ACC: 0.9868864	Validation ACC: 0.72432435
Epoch 51/100	0.31s	Loss: 0.6660445	ACC: 0.9935137	Validation ACC: 0.734054
Epoch 52/100	0.31s	Loss: 0.31190708	ACC: 0.9975621	Validation ACC: 0

In [0]:
pred_df = clf_model.transform(test_data)

In [0]:
pred_df.select('target','prediction.result').show()

+------+------+
target|result|
+------+------+
 1| [1]|
 0| [1]|
 1| [1]|
 1| [0]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 0| [1]|
 0| [0]|
 1| [1]|
 1| [1]|
 0| [1]|
 0| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [0]|
 1| [1]|
 0| [0]|
+------+------+
only showing top 20 rows

In [0]:
preds_df = pred_df.select('target','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))


In [0]:
preds_df

,target,result
0,1,1
1,0,1
2,1,1
3,1,0
4,1,1
5,1,1
6,1,1
7,1,1
8,0,1
9,0,0


In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(preds_df['result'], preds_df['target'], digits=4))

print (accuracy_score(preds_df['result'], preds_df['target']))


precision recall f1-score support

 0 0.4767 0.5018 0.4889 570
 1 0.8321 0.8175 0.8247 1721

 micro avg 0.7390 0.7390 0.7390 2291
 macro avg 0.6544 0.6597 0.6568 2291
weighted avg 0.7436 0.7390 0.7412 2291

0.7389786119598428

# Case Study: Alexa Review Classification

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/amazon_alexa.tsv

In [0]:
import pandas as pd
df = pd.read_csv('amazon_alexa.tsv', sep='\t')
df

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer a question correctly but Alexa says you got it wr...",1
3,5,31-Jul-18,Charcoal Fabric,"I have had a lot of fun with this thing. My 4 yr old learns about dinosaurs, i control the light...",1
4,5,31-Jul-18,Charcoal Fabric,Music,1
5,5,31-Jul-18,Heather Gray Fabric,I received the echo as a gift. I needed another Bluetooth or something to play music easily acce...,1
6,3,31-Jul-18,Sandstone Fabric,"Without having a cellphone, I cannot use many of her features. I have an iPad but do not see tha...",1
7,5,31-Jul-18,Charcoal Fabric,I think this is the 5th one I've purchased. I'm working on getting one in every room of my house...,1
8,5,30-Jul-18,Heather Gray Fabric,looks great,1
9,5,30-Jul-18,Heather Gray Fabric,"Love it! I’ve listened to songs I haven’t heard since childhood! I get the news, weather, inform...",1


In [0]:
df.verified_reviews = df.verified_reviews.str.lower()

In [0]:
df.feedback.value_counts()

In [0]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

column_trans = make_column_transformer(
     (TfidfVectorizer(max_features=1000,  norm='l2', ngram_range=(1, 3)), 'verified_reviews'))

X = column_trans.fit_transform(df.drop(['feedback'], axis=1))

y = df.feedback

In [0]:
import scipy.sparse

sdf = pd.SparseDataFrame(X)

sdf.columns = ['col_{}'.format(i) for i in range(X.shape[1])]

sdf = sdf.fillna(0)

sdf['feedback']= y

sdf.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,...,col_951,col_952,col_953,col_954,col_955,col_956,col_957,col_958,col_959,col_960,col_961,col_962,col_963,col_964,col_965,col_966,col_967,col_968,col_969,col_970,col_971,col_972,col_973,col_974,col_975,col_976,col_977,col_978,col_979,col_980,col_981,col_982,col_983,col_984,col_985,col_986,col_987,col_988,col_989,col_990,col_991,col_992,col_993,col_994,col_995,col_996,col_997,col_998,col_999,feedback
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.145963,0.146283,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.327234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.324472,0.0,0.149662,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.16371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.104466,0.0,0.0,0.0,0.0,0.0,0.210616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1


In [0]:
input_spark_df = spark.createDataFrame(sdf)

In [0]:
input_spark_df.show(5)

+-----+-----+-----+-----+-----+-----+-----+-------------------+-------------------+-------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------------------+------+------+------+------+------+------+------+------+------+------+-------------------+------+------+------+------+------+------+------+------+------+------+-------------------+-------------------+------+------+------+------+------+------+------+-------------------+------+------+------+------+------+------+-------------------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------------------+-------------------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------------------+-------+-------+-------+-------------------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------------------+------------------+-------+-------+-------+-------+-------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------------------+-------+-------+-------------------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------------------+-------------------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+---

In [0]:

(training_data, test_data) = input_spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 2490
Test Dataset Count: 660

In [0]:
%sh cd file:/databricks/driver/gc_graph && ls -la

total 144
drwxr-xr-x 2 root root 4096 Nov 1 13:11 .
drwxr-xr-x 3 root root 4096 Nov 1 12:03 ..
-rw-r--r-- 1 root root 135651 Nov 1 13:11 gcl.302.2.pb

In [0]:
from sparknlp_jsl.base import *

features_asm = FeaturesAssembler()\
      .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
      .setOutputCol("features")
        
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("feedback")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")\
    .setModelFile("/databricks/driver/gc_graph/pet.in1202D.out2.pb").setEpochsNumber(50)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.001)\
    .setOutputLogsPath("logs")\

clf_Pipeline = Pipeline(stages=[
    features_asm, 
    gen_clf])

clf_model = clf_Pipeline.fit(training_data)


In [0]:
pred_df = clf_model.transform(test_data)

preds_df = pred_df.select('feedback','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(preds_df['result'], preds_df['feedback'], digits=4))

print (accuracy_score(preds_df['result'], preds_df['feedback']))


precision recall f1-score support

 0 0.5962 0.4844 0.5345 64
 1 0.9457 0.9648 0.9551 596

 micro avg 0.9182 0.9182 0.9182 660
 macro avg 0.7709 0.7246 0.7448 660
weighted avg 0.9118 0.9182 0.9144 660

0.9181818181818182